## Imports

In [1]:
import librosa
import spotipy
import os, requests, time, random

import pandas as pd
import numpy as np

from src.obtain.spotify_metadata import generate_token, download_playlist_metadata
from src.vinyl.build_datasets import extract_features
from src.vinyl.build_datasets import build_dataset
import src.vinyl.db_manager as crates

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import logging
import keras
from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers import Dense
from keras.optimizers import Adam

%matplotlib inline
import matplotlib.pyplot as plt
import librosa.display
import IPython.display as ipd

Using TensorFlow backend.


# Load New Data, Retrain and Re-evaluate


## Specify Model

In [2]:
features_dict = {
    librosa.feature.mfcc : {'n_mfcc':12},
    librosa.feature.spectral_centroid : {},
    librosa.feature.chroma_stft : {'n_chroma':12},
    librosa.feature.spectral_contrast : {'n_bands':6},
    #librosa.feature.tempogram : {'win_length':192}
}

model_save_path = "models/zouk_classifier_spectral_LSTM3.h5"

## Load Positive Samples

In [3]:
zouk_songs = crates.get_playlist_songs('zoukables')
zouk_features_path = "data/zoukable_spectral.npy"
zouk_data = np.load(zouk_features_path)

## Load Negative Samples

In [4]:
non_zouk_songs = crates.sample_other_songs(n_songs=len(zouk_songs), skip_genres=["zoukables"])
non_zouk_data = build_dataset(non_zouk_songs, features_dict)

In [6]:
target = np.array([1] * len(zouk_songs) + [0] * len(non_zouk_songs))

## Build Train/Test Data Sets

In [7]:
X = np.concatenate((zouk_data, non_zouk_data))

train_idx, test_idx, y_train, y_test = train_test_split(
    range(X.shape[0]), target, test_size=0.33, random_state=42, stratify=target)

X_train = X[train_idx,:,:]
X_test = X[test_idx,:,:]

## Load and Retrain Model

In [8]:
print("Training ...")
batch_size = 35  # num of training examples per minibatch
num_epochs = 400

model = keras.models.load_model(model_save_path)

model.fit(X_train, y_train, batch_size=batch_size, 
          epochs=num_epochs, validation_split=.25, verbose=1,
          callbacks=[
              keras.callbacks.EarlyStopping(patience=8, verbose=1, restore_best_weights=True),
              keras.callbacks.ReduceLROnPlateau(factor=.5, patience=3, verbose=1),])

model.save(model_save_path)

Training ...




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 597 samples, validate on 200 samples
Epoch 1/400
597/597 [==============================] - 189s 317ms/step - loss: 0.4649 - acc: 0.7755 - val_loss: 0.4498 - val_acc: 0.7950
Epoch 2/400
597/597 [==============================] - 179s 300ms/step - loss: 0.4695 - acc: 0.7789 - val_loss: 0.4496 - val_acc: 0.7950
Epoch 3/400
597/597 [==============================] - 183s 306ms/step - loss: 0.4950 - acc: 0.7722 - val_loss: 0.4494 - val_acc: 0.7950
Epoch 4/400
597/597 [==============================] - 178s 297ms/step - loss: 0.4711 - acc: 0.7806 - val_loss: 0.4495 - val_acc: 0.7950
Epoch 5/400
597/597 [==============================] - 176s 295ms/step - loss: 0.4857 - acc: 0.7806 - val_loss: 0.4495 - val_acc: 0.7950
Epoch 6/400
597/597 [====================

## Evaluate Retrained Model

In [9]:
print("\nTesting ...")
score, accuracy = model.evaluate(
    X_test, y_test, batch_size=batch_size, verbose=1
)
print("Test loss:  ", score)
print("Test accuracy:  ", accuracy)


Testing ...
393/393 [==============================] - 26s 67ms/step
Test loss:   0.49997260870824334
Test accuracy:   0.7760814379796424


### Test Accuracy Scores

| Test Run | LSTM3 (spec, temp) | LSTM3 (spec) | LSTM3 (spec, mfcc12) |
|----------|-----|-----|-----|
| 1 | 0.7806 | 0.8092 | 0.8422 |
| 2 | 0.7398 | 0.8117 | 0.7812 |
| 3 | 0.6990 | 0.8168 | 0.8230 |
| 4 | 0.7730 | 0.7710 | 0.7761 |
| 5 | 0.7397 |   |   |


## Inspect Predictions

In [10]:
y_pred = model.predict(X_test)

In [11]:
y_pred_bool = y_pred > 0.65
print(classification_report(y_test, y_pred_bool))

              precision    recall  f1-score   support

           0       0.81      0.72      0.76       197
           1       0.75      0.83      0.79       196

    accuracy                           0.78       393
   macro avg       0.78      0.78      0.78       393
weighted avg       0.78      0.78      0.78       393



In [12]:
all_songs = pd.DataFrame({'song_id':zouk_songs + non_zouk_songs,
                          'target':target})

trainers = all_songs.iloc[train_idx,:].reset_index()

sample0 = trainers[trainers.target==0].sample(10).index
sample1 = trainers[trainers.target==1].sample(10).index
sample_idx = sample0.append(sample1)
samples = trainers.loc[sample_idx]

In [13]:
y_pred = model.predict(X_train[sample_idx,:])
y_pred_bool = y_pred > 0.75
samples['prediction'] = y_pred_bool.astype(int)
print(classification_report(samples.target, y_pred_bool))

              precision    recall  f1-score   support

           0       0.90      0.90      0.90        10
           1       0.90      0.90      0.90        10

    accuracy                           0.90        20
   macro avg       0.90      0.90      0.90        20
weighted avg       0.90      0.90      0.90        20



### Listen to False Positives, False Negatives

In [14]:
fp_index = samples[(samples.target==0) & (samples.prediction==1)].index
fn_index = samples[(samples.target==1) & (samples.prediction==0)].index

print("False Positives:")
for i in fp_index:
    song_id = samples['song_id'][i]
    filepath = crates.get_preview_mp3(song_id)
    print(crates.load_song_metadata(song_id)['title'])
    ipd.display(ipd.Audio(filepath))

print("~" * 32)

print("False Negatives:")
for i in fn_index:
    song_id = samples['song_id'][i]
    filepath = crates.get_preview_mp3(song_id)
    print(crates.load_song_metadata(song_id)['title'])
    ipd.display(ipd.Audio(filepath))


False Positives:
Brandi Wuz Here


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
False Negatives:
Best Part (feat. H.E.R.)


# References

- [Keras docs](https://keras.io/)
- [Librosa docs](https://librosa.github.io/librosa/index.html)
- [Spotipy docs](https://spotipy.readthedocs.io)
- [ruohoruotsi: LSTM Music Genre Classification on GitHub](https://github.com/ruohoruotsi/LSTM-Music-Genre-Classification)
- [Music Genre classification using a hierarchical Long Short Term Memory (LSTM) Model](http://www.cs.cuhk.hk/~khwong/p186_acm_00_main_lstm_music_rev5.pdf)
- [Using CNNs and RNNs for Music Genre Recognition](https://towardsdatascience.com/using-cnns-and-rnns-for-music-genre-recognition-2435fb2ed6af) [(GitHub)](https://github.com/priya-dwivedi/Music_Genre_Classification)